1. Здесь идет установка **необходимых библиотек**

In [ ]:
!pip3 install pydub
!pip3 install vosk
!pip3 install torch
!pip3 install transformers
!git clone https://git.ffmpeg.org/ffmpeg.git ffmpeg
!pip install moviepy

1.2 Vosk model installation

In [ ]:
!curl -o ./model.zip https://alphacephei.com/vosk/models/vosk-model-ru-0.42.zip
!unzip model.zip
!mv vosk-model-ru-0.42/ model
!rm -rf model.zip

This model can be put to VOSK, but its use takes too much resources of the computer itself. Deals with punctuation marks

In [ ]:
!curl -o recasepunc.zip https://alphacephei.com/vosk/models/vosk-recasepunc-ru-0.22.zip
!unzip recasepunc.zip
!mv vosk-recasepunc-ru-0.22/ recasepunc
!rm -rf recasepunc.zip

In [7]:
#import libraries
from vosk import Model, KaldiRecognizer, SetLogLevel
from pydub import AudioSegment
import subprocess
import json
import os
from moviepy.editor import VideoFileClip
from moviepy.video.io.html_tools import ffmpeg_parse_infos


In [ ]:
from moviepy.editor import VideoFileClip
def convert_mp4_to_wav(input_file, output_file):
    video = VideoFileClip(input_file)
    audio = video.audio
    audio.write_audiofile(output_file)

input_file = 'video.mp4'
output_file = 'file.wav'
convert_mp4_to_wav(input_file, output_file)


In [9]:
#SetLogLevel(0)
import os
# checking model
if not os.path.exists("model"):
    print ("Please download the model from https://alphacephei.com/vosk/models and unpack as 'model' in the current folder.")
    exit (1)

# set Frame Rate
FRAME_RATE = 16000
CHANNELS=1

model = Model("model")
rec = KaldiRecognizer(model, FRAME_RATE)
rec.SetWords(True)

# Используя библиотеку pydub делаем предобработку аудио
mp3 = AudioSegment.from_mp3('file.wav')
mp3 = mp3.set_channels(CHANNELS)
mp3 = mp3.set_frame_rate(FRAME_RATE)


In [ ]:
# Преобразуем вывод в json
rec.AcceptWaveform(mp3.raw_data)
result = rec.Result()
text = json.loads(result)["text"]

# Добавляем пунктуацию (Занимает очень много ресурсов, можно и без нее)
#cased = subprocess.check_output('python3 recasepunc/recasepunc.py predict recasepunc/checkpoint', shell=True, text=True, input=text)


In [ ]:
# Записываем результат в файл "data.txt"
with open('ff.txt', 'w') as f:
    json.dump(text, f, ensure_ascii=False, indent=4)